# Modelo Hull-White

## Ecuaciones del Modelo

El modelo de Hull - White es una extensión al modelo de Vasicek. La ecuación diferencial estocástica para la evolución de la tasa instantánea en la medida ajustada por riesgo es:

$$
\begin{equation}
dr_t=\left(\theta_t-\gamma^* r_t\right)dt+\sigma dX_t
\end{equation}
$$

La función $\theta_t$, que se puede interpretar como un nivel de reversión a la media variable en el tiempo, se calibra para ajustar perfectamente el valor de mercado de los bonos cupón cero.

Después de eso, quedan aún dos grados de libertad, $\gamma^*$ y $\sigma$, para calibrar la estructura de volatilidades.

### Fórmula para $\theta_t$

La función $\theta_t$, que permite obtener los precios de mercado de los bonos cupón cero está dada por:

$$
\begin{equation}
\theta_t=\frac{\partial f\left(0,t\right)}{\partial t}+\gamma^*f\left(0,t\right)+\frac{\sigma^2}{2\gamma^*}\left[1-\exp\left(-2\gamma^* T\right)\right]
\end{equation}
$$

Recordar que:

$$
\begin{equation}
f(t,T)=-\frac{\partial\log P(t,T)}{\partial T} \\
P(t,T)=\exp\left[-r\left(t,T\right)\left(T-t\right)\right]
\end{equation}
$$

por lo tanto,

$$
\begin{equation}
-\log P\left(t,T\right)=r\left(t,T\right)\left(T-t\right) \\
-\frac{\partial \log P\left(t,T\right)}{\partial T}=\frac{\partial r\left(t,T\right) }{\partial T}\left(T-t\right)+r\left(t,T\right)
\end{equation}
$$

finalmente,

$$
\frac{\partial f\left(0,t\right)}{\partial t}=\frac{\partial^2 r\left(0,t\right) }{\partial t^2}t+2\frac{\partial r\left(0,t\right) }{\partial t}
$$

### Fórmulas para Bonos Cero Cupón

La fórmula para la valoración de bonos cero cupón en el modelo de Hull - White es la siguiente:

$$
\begin{equation}
Z\left(r_t,t,T\right)=\exp\left[A\left(t,T\right)-B\left(t,T\right)r_t\right]
\end{equation}
$$

$$
\begin{equation}
B\left(t,T\right)=\frac{1}{\gamma^*}\left[1-\exp\left(-\gamma^* T\right)\right)]
\end{equation}
$$

$$
\begin{equation}
A\left(t,T\right)=\log\frac{Z\left(r_0,0,T\right)}{Z\left(r_0,0,t\right)}+B\left(t,T\right)f\left(0,t\right)-\frac{\sigma^2}{4\gamma^*}B\left(t,T\right)^2\left[1-\exp\left(-2\gamma^* T\right)\right)]
\end{equation}
$$

### Fórmulas para Calls y Puts sobre Bonos Cupón Cero

Son las mismas que para el modelo de Vasicek, para una call:

$$
\begin{equation}
Call\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(d_1\right)-KN\left(d_2\right)\right]
\end{equation}
$$


$$
\begin{equation}
d_1=\frac{\log\left(\frac{Z\left(r_0,0,T_B\right)}{KZ\left(r_0,0,T_O\right)}\right)+\frac{1}{2}S_Z\left(T_O\right)^2}{\sqrt{S_Z\left(T_O\right)}}
\end{equation}
$$


$$
\begin{equation}
d_2=d_1-S_Z\left(T_O\right)
\end{equation}
$$


$$
\begin{equation}
S_Z\left(T_O\right)=B\left(T_O,T_B\right)\sqrt{\frac{\sigma^2}{2\gamma^*}\left(1-\exp\left(-2\gamma^* T_O\right)\right)}
\end{equation} 
$$

Y para una put:

$$
\begin{equation}
Put\left(r_0,0\right)=Z\left(r_0,0,T_O\right)\left[KN\left(-d_2\right)-\frac{Z\left(r_0,0,T_B\right)}{Z\left(r_0,0,T_O\right)}N\left(-d_1\right)\right]
\end{equation}
$$

y los mismos valores anteriores para $d_1$ y $d_2$.

## Implementación del Modelo

In [184]:
from finrisk import QC_Financial_3 as Qcf
from scipy.interpolate import interp1d
from modules import auxiliary as aux
from numpy import random as rnd
from typing import List, Tuple
import plotly.express as px
import pandas as pd
import numpy as np
import math

In [2]:
frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'}

Veamos la construcción de la función Theta, la simulación y el cálculo de los factores de descuento con las fórmulas del modelo.

### Función $\theta$

Se obtienen los valores de la curva cero cupón.

In [131]:
curva = pd.read_excel('data/20201012_built_sofr_zero.xlsx')

In [132]:
curva.tail().style.format(frmt)

,plazo,tasa,df
28,7306,0.9253%,83.092401%
29,9133,0.9657%,78.533775%
30,10957,0.9881%,74.332619%
31,14610,0.9408%,68.619685%
32,18262,0.8464%,65.475678%


Por comodidad reescribamos la fórmula para la función $\theta_{t}$:

$$
\begin{equation}
\theta_{t}=\frac{\partial f(0,t)}{\partial t}+\gamma^*f(0,t)+\frac{\sigma^2}{2\gamma^*}\left[1-\exp(-2\gamma^*t)\right]
\end{equation}
$$

Necesitamos una representación en tiempo contínuo de la curva para poder calcular $f(0,t)$ y su derivada. Para eso, vamos a utilizar el objeto `interp1d` de `scipy.interpolate` que permite definir las abcisas de la curva como $t\in\mathbb{R}$.

In [20]:
curva['t'] = curva['plazo'] / 365.0

In [75]:
curva['rate'] = (1 / curva['df'])**(365.0 / curva['plazo']) - 1

A continuación, se construye el objeto:

In [106]:
zcurva = interp1d(curva['t'], curva['rate'], kind='cubic',
                  fill_value="extrapolate")

Veamos un gráfico:

In [133]:
steps_per_year = 365
years = 20
dt = 1 / steps_per_year
result = []
for i in range(0, steps_per_year * years):
    result.append((i * dt, zcurva(i * dt)))
df_result = pd.DataFrame(result, columns=['plazo', 'tasa'])

In [108]:
fig = px.line(
    df_result,
    x='plazo',
    y='tasa',
    title=f'Curva Interpolada'
)
fig.show()

Se definen 3 funciones `float -> float` para calcular el valor de la tasa, su derivada y su derivada segunda en un tiempo $t>0$. Las dos derivadas se calculan desde la derecha.

In [109]:
def zrate(t: float) -> float:
    return zcurva(t)


def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

In [135]:
t = 2
print(f'zrate({t:.2f}) = {zrate(t):.4%}')
print(f'dzrate({t:.2f}) = {dzrate(t):.4%}')
print(f'd2zrate({t:.2f}) = {d2zrate(t):.4%}')

zrate(2.00) = 0.0568%
dzrate(2.00) = -0.0066%
d2zrate(2.00) = 0.0584%


Sabemos que:

$$f(0,t)=r(0,t)+t\frac{\partial r(0,t)}{\partial t}$$,

y también que:

$$\frac{\partial f(0,t)}{\partial t}=2\frac{\partial r(0,t)}{\partial t}+t\frac{\partial^2r(0,t)}{\partial t^2}$$

Podemos ahora definir las funciones `fwd(0,t)` y `dfwd(0,t)`.

In [136]:
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [137]:
print(f'fwd({t:.2f}) = {fwd(t):.4%}')
print(f'dfwd({t:.2f}) = {dfwd(t):.4%}')

fwd(2.00) = 0.0436%
dfwd(2.00) = 0.1037%


Con esto, podemos ya escribir una expresión para la función Theta. Definamos un valor para $\sigma$ y $\gamma^*$:

In [138]:
sigma = .015
gamma = .5
r0 = zrate(0)

In [140]:
def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

In [141]:
print(f'theta({t:.2f}) = {theta(t):.4%}')

theta(2.00) = 0.1449%


Grafiquemos la función:

In [142]:
thetas = []
dt = 1 / 365.0
for i in range(0, 366):
    t = i * dt
    thetas.append((i * dt, theta(t)))
df_theta = pd.DataFrame(thetas, columns=['t', 'theta'])

In [143]:
fig = px.line(
    df_theta,
    x='t',
    y='theta',
    title=f'Función theta(t)'
)
fig.show()

### Simulación

Finalmente, podemos simular.

In [172]:
def sim_hw(gamma: float, sigma: float, theta: float, r0: float,
           num_steps: int, seed = None) -> List[Tuple[float, float]]:
    r = r0
    dt = 1 / 264.0
    sqdt = math.sqrt(dt)
    sim = [(0, r0)]
    rnd.seed(seed)
    for i in range(1, num_steps + 1):
        # print(str(r))
        epsilon = rnd.normal()
        r = r + (theta((i - 1) * dt) - gamma * r) * dt + sigma * sqdt * epsilon
        sim.append((i * dt, r))
        # print(str(theta((i - 1) * dt)) + "\t" + str(epsilon) + "\t" + str(r) )
    return sim

In [173]:
dt = 1 / 264.0
sim = sim_hw(gamma, sigma, theta, r0, 264, 1000)

In [174]:
df_sim = pd.DataFrame(sim, columns=['t', 'tasa'])

In [175]:
fig = px.line(
    df_sim,
    x='t',
    y='tasa',
    title=f'Simulación Hull-White'
)
fig.show()

In [176]:
print(f'plazo: {sim[263][0]:.4f},\ntasa: {sim[263][1]:.4%}')

plazo: 0.9962,
tasa: -1.8371%


### Bono Cupón Cero

Función $B$ de la fórmula para un bono cupón cero:

In [125]:
def b_hw(gamma: float, t: float, T: float) -> float:
    """
    gamma : para 
    sigma: 
    t : plazo de....
    T: plazo ....
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

Función $A$ de la fórmula para un bono cupón cero:

In [156]:
def a_hw(zrate: float, fwd: float, gamma: float, sigma: float, t: float, T: float, verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T)
    dft = math.exp(-zrate(t) * t)
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

Factor de descuento según HW.

In [157]:
def zero_hw(r: float, gamma: float, sigma: float,
            zrate: float, fwd: float, t: float, T: float) -> float:
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)

Verifiquemos que los valores que entrega la fórmula de HW coinciden con los datos de la curva:

In [168]:
r0 = zrate(0)
print(f"r0:        {r0:.8%}")
t = 0
T = 1.1234
z = zero_hw(r0, gamma, sigma, zrate, fwd, t, T)
print(f"df hw:    {z:.8%}")
print(f"df curva: {math.exp(-zrate(T) * T):.8%}")

r0:        0.07884508%
df hw:    99.92235606%
df curva: 99.92235606%


### Ejercicio de Valorización

Vamos a valorizar, en 1Y más, un bono semestral a tasa fija del 5% que en 1Y más le quedarán 4Y. Podemos además hacer el ejercicio de verificar la simulación calculando los valores de los bonos cero cupón usando la simulación.

Utilizaremos la siguiente función:

In [169]:
def bono_tasa_fija(start_time: float, yf: float,
                   num_cupones: int, valor_tasa: float) -> List[Tuple[float, float]]:
    """
    Retorna los plazos y flujos de un bono a tasa fija bullet con nominal = 0.
    
    params:
    
    - start_time: fecha (expresada en fracción de año) en que comienza el devengo del primer cupón.
    - yf: fracción de año que representa la periodicidad del bono (yf = .5 -> bono semestral).
    - num_cupones: número de cupones del bono
    - valor_tasa: valor de la tasa fija del bono. Los intereses se calculan de forma lineal.
    
    return:
    
    - Una `list` de `tuple` con la fecha de pago del cupón (como instante de tiempo) y el monto del cupón.
    """
    result = []
    flujo = valor_tasa * yf
    for i in range(1, num_cupones + 1):
        if i == num_cupones:
            flujo += 1.0
        result.append((i * yf + start_time, flujo))
    return result

In [171]:
start_time = 1.5
yf = .5
tasa = .05
num_cupones = 8
bf = bono_tasa_fija(start_time, yf, num_cupones, tasa)
bf

[(2.0, 0.025),
 (2.5, 0.025),
 (3.0, 0.025),
 (3.5, 0.025),
 (4.0, 0.025),
 (4.5, 0.025),
 (5.0, 0.025),
 (5.5, 1.025)]

In [178]:
r1Y = sim[264][1]
print(f'Tasa instantánea simulada a 1Y: {r1Y:.6%}')

Tasa instantánea simulada a 1Y: -1.906778%


¿Cuál es la curva de factores de descuento en t = 1Y en esta simulación?

In [181]:
t = 1
zeros_1Y = []
for plazo in [b[0] for b in bf]:
    zeros_1Y.append(zero_hw(r1Y, gamma, sigma, zrate, fwd, t, plazo))
print(zeros_1Y)

[1.0151305857938773, 1.020094537621821, 1.0235079895437473, 1.025567030063376, 1.026446416893182, 1.0262951649757672, 1.0251582633566996, 1.0230557617338862]


¿Cuál es el valor presente?

In [183]:
vp = np.dot(zeros_1Y, [b[1] for b in bf])
print(f"vp: {vp:.6%}")

vp: 122.768716%


Chequear que el operador `np.dot` haga lo que creemos que está haciendo:

In [186]:
vp1 = 0
for y in zip([b[1] for b in bf], zeros_1Y):
    vp1 += y[0] * y[1]
print(f"vp1: {vp1:.6%}")

vp1: 122.768716%


### No Terminado

In [120]:
def sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed = None):
    dt = 1/264.0
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)
    # for i in range(0, 5000):
    #    rnd.normal()
    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    for i in range(0, num_steps):
        theta_array[i] = theta(i * dt)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            # print(str(r))
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
            # print(str(theta_array[j - 1]) + "\t" + str(alea[i][j - 1]) + "\t" + str(r) )
    return sim

In [28]:
num_sim = 10
def sim_many(num_sim, r0, gamma, sigma, zrate, fwd, theta, bf):
    result = []
    for i in range(0, num_sim):
        r1y = sim_hw(gamma, sigma, theta, r0, 264)[263]
        zeros_1Y = []
        for plazo in bf[0]:
            zeros_1Y.append(zero_hw(r1y, gamma, sigma, zrate, fwd, t, plazo))
        vp = np.dot(zeros_1Y, bf[1])
        result.append(vp)
    return result

res = sim_many(num_sim, r0, gamma, sigma, zrate, fwd, theta, bf)
res

[1.1279540277552287,
 1.1017071334219528,
 1.109978338654758,
 1.129794026792734,
 1.1089066369872622,
 1.1037275422507045,
 1.090911338993188,
 1.1169754223503474,
 1.1028002001274457,
 1.1181127285470778]

In [29]:
num_sim = 1000
# dfs = 0
t = .25
sim = 0
seed = None
num_steps = int(t*264)
print("num_steps: " + str(num_steps))
for i in range(0, num_sim):
    sim += math.exp(np.sum(- dt * np.array(
        sim_hw(gamma, sigma, theta, r0, num_steps, seed))))
    # dfs += math.exp(np.sum(sim))
ez = sim / num_sim
z_curva = math.exp(-zrate(t) * t)
print("ez: " + str(ez))
print("z_curva: " + str(z_curva))

num_steps: 66
ez: 0.997597204287458
z_curva: 0.9976549482230134


In [31]:
num_sim = 50000
t = .25
num_steps = int(t * 264)
print("num_steps: " + str(num_steps))
seed = None
df = 0
s = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
for sim in s:
    # print(len(sim))
    df += math.exp(-dt * np.sum(sim))
ez = df / num_sim
z_curva = math.exp(-zrate(t) * t)
print("ez: " + str(ez))
print("z_curva: " + str(z_curva))
print(t)

num_steps: 66
ez: 0.9976520059005399
z_curva: 0.9976549482230134
0.25
